In [53]:
# !pip install simpy

In [54]:
import simpy
import random

In [55]:
# Tasa de solicitudes por segundo
LAMBDA = 2400 / 60  # Convertimos las solicitudes por minuto a solicitudes por segundo

# Duración de la simulación en segundos (1 hora)
SIMULATION_TIME = 3600  

# Variables para estadísticas para Proveedor 1 y Proveedor 2
total_requests_1 = total_requests_2 = 0
total_time_in_queue_1 = total_time_in_queue_2 = 0
total_time_of_service_1 = total_time_of_service_2 = 0

In [56]:
def request_generator(env, mu, server, provider):
    global total_requests_1, total_requests_2
    while True:
        yield env.timeout(random.expovariate(LAMBDA))
        if provider == 1:
            total_requests_1 += 1
            env.process(request(env, mu, server, provider))
        elif provider == 2:
            total_requests_2 += 1
            env.process(request(env, mu, server, provider))

In [57]:
def request(env, mu, server, provider):
    global total_time_in_queue_1, total_time_of_service_1, total_time_in_queue_2, total_time_of_service_2
    
    arrival_time = env.now
    with server.request() as req:
        yield req
        wait_time = env.now - arrival_time
        
        if provider == 1:
            total_time_in_queue_1 += wait_time
        elif provider == 2:
            total_time_in_queue_2 += wait_time
        
        # El servicio se modela con una distribución exponencial
        yield env.timeout(random.expovariate(mu))
        
        if provider == 1:
            total_time_of_service_1 += env.now - arrival_time
        elif provider == 2:
            total_time_of_service_2 += env.now - arrival_time

In [58]:
env = simpy.Environment()
server_1 = simpy.Resource(env, capacity=1)
server_2 = simpy.Resource(env, capacity=SERVERS_2)  # Asumimos 10 servidores

env.process(request_generator(env, MU_1, server_1, 1))  # El 1 al final indica Proveedor 1
env.process(request_generator(env, MU_2, server_2, 2))  # El 2 al final indica Proveedor 2

env.run(until=SIMULATION_TIME)

In [59]:
# Proveedor 1
MU_1 = 100  # Tasa de servicio de 100 solicitudes por segundo

# Proveedor 2
MU_2 = 10    # Tasa de servicio de 10 solicitudes por segundo
SERVERS_2 = 10  # Número de servidores

# Al finalizar la simulación, calculamos y mostramos estadísticas
idle_time_1 = SIMULATION_TIME - total_time_of_service_1
idle_time_2 = SIMULATION_TIME - total_time_of_service_2

# a. ¿Cuántas solicitudes atendió cada servidor?
print(f"Solicitudes totales atendidas por el Proveedor 1: {total_requests_1}")
print(f"Solicitudes totales atendidas por el Proveedor 2: {total_requests_2}")

# b. ¿Cuánto tiempo estuvo cada servidor ocupado?
print(f"Tiempo total de servicio del Proveedor 1: {total_time_of_service_1:.2f} segundos")
print(f"Tiempo total de servicio del Proveedor 2: {total_time_of_service_2:.2f} segundos")

# c. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?
print(f"Tiempo total desocupado para el Proveedor 1: {idle_time_1:.2f} segundos")
print(f"Tiempo total desocupado para el Proveedor 2: {idle_time_2:.2f} segundos")

# d. ¿Cuánto tiempo en total estuvieron las solicitudes en cola?
print(f"Tiempo total en cola para el Proveedor 1: {total_time_in_queue_1:.2f} segundos")
print(f"Tiempo total en cola para el Proveedor 2: {total_time_in_queue_2:.2f} segundos")

# e. En promedio, ¿cuánto tiempo estuvo cada solicitud en cola?
print(f"Tiempo promedio en cola para el Proveedor 1: {total_time_in_queue_1/total_requests_1:.2f} segundos")
print(f"Tiempo promedio en cola para el Proveedor 2: {total_time_in_queue_2/total_requests_2:.2f} segundos")

# f. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?
print(f"Número promedio de solicitudes en cola por segundo para el Proveedor 1: {total_time_in_queue_1/SIMULATION_TIME:.2f}")
print(f"Número promedio de solicitudes en cola por segundo para el Proveedor 2: {total_time_in_queue_2/SIMULATION_TIME:.2f}")


Solicitudes totales atendidas por el Proveedor 1: 143817
Solicitudes totales atendidas por el Proveedor 2: 143861
Tiempo total de servicio del Proveedor 1: 2414.35 segundos
Tiempo total de servicio del Proveedor 2: 14.43 segundos
Tiempo total desocupado para el Proveedor 1: 1185.65 segundos
Tiempo total desocupado para el Proveedor 2: 3585.57 segundos
Tiempo total en cola para el Proveedor 1: 975.21 segundos
Tiempo total en cola para el Proveedor 2: 0.00 segundos
Tiempo promedio en cola para el Proveedor 1: 0.01 segundos
Tiempo promedio en cola para el Proveedor 2: 0.00 segundos
Número promedio de solicitudes en cola por segundo para el Proveedor 1: 0.27
Número promedio de solicitudes en cola por segundo para el Proveedor 2: 0.00
